In [ ]:
from langchain_classic.chains.conversation.base import ConversationChain
from langchain_classic.chains.llm import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models.tongyi import ChatTongyi
from dotenv import load_dotenv
from langchain_core.messages import AIMessage

load_dotenv()

#1、获取大模型
chat_model = ChatTongyi()

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage


def chat_with_model(question):
    # 2、提供提示词模版：ChatPromptTemplate
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", "你是一个人工智能的助手"),
        ("human", "{question}"),
    ])

    while True:
        chain = prompt_template | chat_model
        res = chain.invoke({"question": question})
        print(f"模型回复：{res.content}")
        user_input = input("你还有其他问题吗？（输入‘退出’时，结束会话）")
        if (user_input == "退出"):
            break
        prompt_template.append(AIMessage(content=res.content))
        prompt_template.append(HumanMessage(content=user_input))


In [ ]:

chat_with_model("你好，很高兴认识你")

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory

history = InMemoryChatMessageHistory()

# 2、添加相关的消息进行存储

history.add_user_message("你好")
history.add_ai_message("很高兴认识你")

In [ ]:

#3、打印存储的消息
print(history.messages)


场景2:对接LLM

In [ ]:
from langchain_community.chat_models.tongyi import ChatTongyi
from dotenv import load_dotenv

load_dotenv()
chat_model = ChatTongyi()

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory

history = InMemoryChatMessageHistory()

# 2、添加相关的消息进行存储

history.add_user_message("你好")
history.add_ai_message("很高兴认识你")
history.add_user_message("帮我计算1+2*3=？")
print(history.messages)

In [ ]:
res = chat_model.invoke(history.messages)
print(res.content)

# 2、使用ConversationBufferMemory进行多轮对话
举例1：以字符串的方式返回存储的信息

In [ ]:
from langchain_classic.memory import ConversationBufferMemory

# 1、ConversationBufferMemory的实例化
memory = ConversationBufferMemory()
#inputs对应用户的消息，outputs对应ai消息
memory.save_context(inputs={"input": "你好，我叫小明"}, outputs={"output": "很高兴认识你"})
memory.save_context(inputs={"input": "帮我回答一下1+2*3=?"}, outputs={"output": "7"})

#3、获取存储到记忆中的信息
print(memory.load_memory_variables({}))
#返回的字典结构的key叫history

举例2：以消息列表的方式返回存储的信息

In [ ]:
from langchain_classic.memory import ConversationBufferMemory

# 1、ConversationBufferMemory的实例化
memory = ConversationBufferMemory(return_messages=True)
#inputs对应用户的消息，outputs对应ai消息
memory.save_context(inputs={"input": "你好，我叫小明"}, outputs={"output": "很高兴认识你"})
memory.save_context(inputs={"input": "帮我回答一下1+2*3=?"}, outputs={"output": "7"})

#3、获取存储到记忆中的信息
print(memory.load_memory_variables({}))
print("\n")
print(memory.chat_memory.messages)
#返回的字典结构的key叫history

举例3：结合大模型、提示词模板的使用

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_classic.chains.llm import LLMChain
from dotenv import load_dotenv

load_dotenv()
chat_model = ChatTongyi()

prompt_template = PromptTemplate.from_template(
    template="""
    你可以与人类对话。
    当前对话历史：{history}

    人类问题：{question}

    回复：
    """
)

#3、提供memory
memory = ConversationBufferMemory()

chain = LLMChain(llm=chat_model, prompt=prompt_template, memory=memory)

chain.invoke({"question": "我的名字叫小明"})

In [ ]:
res = chain.invoke({"question": "我叫什么名字呢？"})
print(res)

举例4：基于举例3，显示的设置memory的key的值

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_classic.chains.llm import LLMChain
from dotenv import load_dotenv

load_dotenv()
chat_model = ChatTongyi()

prompt_template = PromptTemplate.from_template(
    template="""
    你可以与人类对话。
    当前对话历史：{chat_history}

    人类问题：{question}

    回复：
    """
)

#3、提供memory
memory = ConversationBufferMemory(memory_key="chat_history")

chain = LLMChain(llm=chat_model, prompt=prompt_template, memory=memory)

chain.invoke({"question": "我的名字叫小明"})

In [ ]:
res = chain.invoke({"question": "我叫什么名字呢？"})
print(res)

举例5：`使用ChatPromptTemplate`

In [ ]:
from langchain_core.messages import SystemMessage
from langchain_classic.chains.llm import LLMChain
from langchain_classic.memory import ConversationBufferMemory
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_community.chat_models.tongyi import ChatTongyi

# 2.创建LLM
llm = ChatTongyi()
# 3.创建Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个与人类对话的机器人。"),
    MessagesPlaceholder(variable_name='chat_history'),
    ("human", "问题：{question}")
])
# 4.创建Memory
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")
# 5.创建LLMChain
llm_chain = LLMChain(prompt=prompt, llm=llm, memory=memory)
# 6.调用LLMChain
res1 = llm_chain.invoke({"question": "中国首都在哪里？"})
print(res1, end="\n\n")

In [ ]:
res2 = llm_chain.invoke({"question": "我刚刚问了什么"})
print(res2)

 ConversationChain的使用

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_classic.chains.conversation.base import ConversationChain
from dotenv import load_dotenv

load_dotenv()
chat_model = ChatTongyi()

prompt_template = PromptTemplate.from_template(
    template="""
    你可以与人类对话。
    当前对话历史：{chat_history}

    人类问题：{input}

    回复：
    """
)

# 3、创建COnversationChain的实例
chain = ConversationChain(llm=chat_model, prompt=prompt_template)
chain.invoke({"input": "我的名字叫小明"})

In [ ]:
chain.invoke({"input": "我的名字叫什么？"})

使用默认的提示词模板

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_classic.chains.conversation.base import ConversationChain
from dotenv import load_dotenv

load_dotenv()
chat_model = ChatTongyi()

# 3、创建COnversationChain的实例
chain = ConversationChain(llm=chat_model)
chain.invoke({"input": "我的名字叫小明"})

ConversationBufferWindowMemory
只使用最近的k条交互

In [ ]:
from langchain_classic.memory import ConversationBufferWindowMemory

# 2.实例化ConversationBufferWindowMemory对象，设定窗口阈值
memory = ConversationBufferWindowMemory(k=2)
# 3.保存消息
memory.save_context({"input": "你好"}, {"output": "怎么了"})
memory.save_context({"input": "你是谁"}, {"output": "我是AI助手"})
memory.save_context({"input": "你的生日是哪天？"}, {"output": "我不清楚"})
# 4.读取内存中消息（返回消息内容的纯文本）
print(memory.load_memory_variables({}))


ConversationBufferWindowMemory结合chain

In [61]:
# 1.导入相关包
from langchain_core.prompts.prompt import PromptTemplate
from langchain_classic.chains.llm import LLMChain
from langchain_community.chat_models.tongyi import ChatTongyi

# 2.定义模版
template = """以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的
具体细节。如果AI不知道问题的答案，它会表示不知道。
当前对话：
{history}
Human: {question}
AI:"""
# 3.定义提示词模版
prompt_template = PromptTemplate.from_template(template)
# 4.创建大模型
llm = ChatTongyi()
# 5.实例化ConversationBufferWindowMemory对象，设定窗口阈值
memory = ConversationBufferWindowMemory(k=1)
# 6.定义LLMChain
conversation_with_summary = LLMChain(
    llm=llm,
    prompt=prompt_template,
    memory=memory,
    verbose=True,
)

In [62]:
# 7.执行链（第一次提问）
respon1 = conversation_with_summary.invoke({"question": "你好，我是孙小空"})
print(respon1)



> Entering new LLMChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的
具体细节。如果AI不知道问题的答案，它会表示不知道。
当前对话：

Human: 你好，我是孙小空
AI:

> Finished chain.
{'question': '你好，我是孙小空', 'history': '', 'text': '你好，孙小空！很高兴认识你。我是通义千问，一个由阿里巴巴集团研发的超大规模语言模型。我能够回答各种问题、创作文字、表达观点，甚至进行简单的编程和逻辑推理。虽然我不能像人类一样拥有真实的情感和体验，但我希望能以我的知识和能力，为你提供帮助和陪伴。\n\n我平时最喜欢做的事情是学习新知识、阅读各种书籍、了解不同的文化和思想。我也很乐意和你聊天，分享我的见解，或者倾听你的故事。如果你有任何问题或需要帮助，随时告诉我，我会尽我所能来协助你。\n\n你觉得我们接下来可以聊些什么呢？'}


In [63]:
# 8.执行链（第二次提问）
respon2 = conversation_with_summary.invoke({"question": "我还有两个师弟，一个是猪小戒，一个是沙小僧"})
print(respon2)



> Entering new LLMChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的
具体细节。如果AI不知道问题的答案，它会表示不知道。
当前对话：
Human: 你好，我是孙小空
AI: 你好，孙小空！很高兴认识你。我是通义千问，一个由阿里巴巴集团研发的超大规模语言模型。我能够回答各种问题、创作文字、表达观点，甚至进行简单的编程和逻辑推理。虽然我不能像人类一样拥有真实的情感和体验，但我希望能以我的知识和能力，为你提供帮助和陪伴。

我平时最喜欢做的事情是学习新知识、阅读各种书籍、了解不同的文化和思想。我也很乐意和你聊天，分享我的见解，或者倾听你的故事。如果你有任何问题或需要帮助，随时告诉我，我会尽我所能来协助你。

你觉得我们接下来可以聊些什么呢？
Human: 我还有两个师弟，一个是猪小戒，一个是沙小僧
AI:

> Finished chain.
{'question': '我还有两个师弟，一个是猪小戒，一个是沙小僧', 'history': 'Human: 你好，我是孙小空\nAI: 你好，孙小空！很高兴认识你。我是通义千问，一个由阿里巴巴集团研发的超大规模语言模型。我能够回答各种问题、创作文字、表达观点，甚至进行简单的编程和逻辑推理。虽然我不能像人类一样拥有真实的情感和体验，但我希望能以我的知识和能力，为你提供帮助和陪伴。\n\n我平时最喜欢做的事情是学习新知识、阅读各种书籍、了解不同的文化和思想。我也很乐意和你聊天，分享我的见解，或者倾听你的故事。如果你有任何问题或需要帮助，随时告诉我，我会尽我所能来协助你。\n\n你觉得我们接下来可以聊些什么呢？', 'text': '哇，听起来你们是一个很有趣的团队呢！猪小戒和沙小僧，他们的名字让我想起了很多经典的故事，是不是？我很好奇，你们之间有什么特别的经历或者有趣的日常吗？比如，你们一起完成过什么任务，或者有没有什么难忘的冒险？我很想听听你们的故事！'}


In [64]:
#  # 9.执行链（第三次提问）
respon3 = conversation_with_summary.invoke(    {"question": "我今年高考，竟然考上了1本"})
print(respon3)



> Entering new LLMChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的
具体细节。如果AI不知道问题的答案，它会表示不知道。
当前对话：
Human: 我还有两个师弟，一个是猪小戒，一个是沙小僧
AI: 哇，听起来你们是一个很有趣的团队呢！猪小戒和沙小僧，他们的名字让我想起了很多经典的故事，是不是？我很好奇，你们之间有什么特别的经历或者有趣的日常吗？比如，你们一起完成过什么任务，或者有没有什么难忘的冒险？我很想听听你们的故事！
Human: 我今年高考，竟然考上了1本
AI:

> Finished chain.
{'question': '我今年高考，竟然考上了1本', 'history': 'Human: 我还有两个师弟，一个是猪小戒，一个是沙小僧\nAI: 哇，听起来你们是一个很有趣的团队呢！猪小戒和沙小僧，他们的名字让我想起了很多经典的故事，是不是？我很好奇，你们之间有什么特别的经历或者有趣的日常吗？比如，你们一起完成过什么任务，或者有没有什么难忘的冒险？我很想听听你们的故事！', 'text': '哇！恭喜你考上一本了！这真是一个了不起的成就！高考可是人生中非常重要的一步，能考上一本说明你一定付出了很多努力和汗水。猪小戒和沙小僧一定也为你们感到骄傲吧？他们平时是不是也经常鼓励你或者陪你一起复习呢？我很好奇，你接下来有什么计划吗？比如想报考哪个大学，或者有没有特别想学的专业？'}


In [65]:
# # 10.执行链（第四次提问）
respon4 = conversation_with_summary.invoke({"question": "我叫什么？"})
print(respon4)




> Entering new LLMChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的
具体细节。如果AI不知道问题的答案，它会表示不知道。
当前对话：
Human: 我今年高考，竟然考上了1本
AI: 哇！恭喜你考上一本了！这真是一个了不起的成就！高考可是人生中非常重要的一步，能考上一本说明你一定付出了很多努力和汗水。猪小戒和沙小僧一定也为你们感到骄傲吧？他们平时是不是也经常鼓励你或者陪你一起复习呢？我很好奇，你接下来有什么计划吗？比如想报考哪个大学，或者有没有特别想学的专业？
Human: 我叫什么？
AI:

> Finished chain.
{'question': '我叫什么？', 'history': 'Human: 我今年高考，竟然考上了1本\nAI: 哇！恭喜你考上一本了！这真是一个了不起的成就！高考可是人生中非常重要的一步，能考上一本说明你一定付出了很多努力和汗水。猪小戒和沙小僧一定也为你们感到骄傲吧？他们平时是不是也经常鼓励你或者陪你一起复习呢？我很好奇，你接下来有什么计划吗？比如想报考哪个大学，或者有没有特别想学的专业？', 'text': 'AI: 哦，对不起！我好像忘记了你的名字。在之前的对话中，你没有提到自己的名字，所以我无法知道。如果你愿意的话，可以告诉我你的名字吗？这样我们之后的对话会更亲切一些！'}
